In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from tpot import TPOTRegressor
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [ ]:
housing = pd.read_csv('/kaggle/input/housesalesprediction/kc_house_data.csv')  

In [ ]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
housing.head()

In [ ]:
housing.shape

In [ ]:
housing['date'] = housing['date'].apply(lambda x: x.split('T')[0])

In [ ]:
X = housing.drop(['id', 'price'], axis=1).values.astype(np.float)
y = housing.iloc[:, 2].values.astype(np.float)

In [ ]:
X.shape

In [ ]:
housing.isnull().any().any()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.66, test_size=0.33, random_state=42)

In [ ]:
tpot = TPOTRegressor(generations=3, population_size=25, verbosity=3)
tpot.fit(X_train, y_train)

In [ ]:
tpot.export('tpot_exported_pipeline.py')

import numpy as np

import pandas as pd

from sklearn.ensemble import ExtraTreesRegressor

from sklearn.linear_model import RidgeCV

from sklearn.model_selection import train_test_split

from sklearn.pipeline import make_pipeline, make_union

from tpot.builtins import StackingEstimator

####### NOTE: Make sure that the outcome column is labeled 'target' in the data file

tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)

features = tpot_data.drop('target', axis=1)

training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['target'], random_state=None)

####### Average CV score on the training set was: -17185500074.45013

exported_pipeline = make_pipeline(
    StackingEstimator(estimator=RidgeCV()),
    ExtraTreesRegressor(bootstrap=True, max_features=1.0, min_samples_leaf=1, min_samples_split=17, n_estimators=100)
)

exported_pipeline.fit(training_features, training_target)

results = exported_pipeline.predict(testing_features)

In [ ]:
print("Training neg_mean_squared_error: {:.4f}".format(tpot.score(X_train, y_train)))
print("Testing neg_mean_squared_error:  {:.4f}".format(tpot.score(X_test, y_test)))

In [ ]:
mean_absolute_error(y_test, tpot.predict(X_test))

In [ ]:
tpot.fitted_pipeline_

**Ingeniería de características (Feature engineering):**

Que limpieza de datos se hizo de forma manual : la columna date tenia un formato string con un T y 0 agregados al fin. Eliminamos esta parte para quedarnos con solo un numero que pudimos convertir a un type float

Columnas descartadas : el id no sirve para predecir el precio de una casa entonces lo eliminas como feature (si bien tenia un impacto sobre el accuracy del modelo seria por overfitting)

Categorías convertidas a rangos : ninguna

Nuevas columnas creadas a partir de otras : ninguna

**Modelo:**

Cual fue el modelo final utilizado para obtener la regresión : extratreesregressor

Que medidas de rendimiento utilizó el algoritmo : para TPOT classifier se usa accuracy pero utilizamops TPOT regressor, y se usa neg_mean_squared_error, que es el numero negative de mean_square_error (sumatoria de las differencias al cuadrado entre valor real y valor predicha de las casas dividido por la cantidad de predicciones)

Que resultado obtuvieron dichas medidas : Training neg_mean_squared_error: -9784258791.3107; Testing neg_mean_squared_error: -22252029797.0977


**Pipeline:**

Cual fue el pipeline sugerido por TPOT : stackingestimator, extratreesregressor

Explicar brevemente que hace cada uno de los métodos ejecutados en
el pipeline y qué tipo de algoritmo es :
StackingEstimator es un meta-transformer para agregar predicciones y/o class probabilidades como synthetic caracteristica(s). Extratreesregressor implementa un metaestimador que se ajusta a una serie de árboles de decisión aleatorios en varias submuestras del conjunto de datos y usa promedios para mejorar la precisión predictiva y controlar el over-fitting.



**Conclusiones:**

In [ ]:
housing['price'].hist()

In [ ]:
housing['price'].min()

In [ ]:
housing['price'].max()

In [ ]:
def number_of_houses_well_predicted(Y_pred, Y):
    nb_houses = 0
    for i in range(0, len(Y_pred)):
        if abs(Y_pred[i] - Y[i]) < Y_pred[i]*0.05:
            nb_houses += 1
    return nb_houses

In [ ]:
number_of_houses_well_predicted(tpot.predict(X_test),y_test)

In [ ]:
number_of_houses_well_predicted(tpot.predict(X_test),y_test)/X.shape[0]

¿Es posible predecir el precio de una propiedad con los datos
 proporcionados? El root mean square error sobre test es 149171, y sobre training es 98915. El mean absolute eroor sobre test es 76443. Este tipo de error estaria muy mal sobre la casa la mas barata (75000 dollars) pero daria una buena approximacion sobre la casa la mas cara (7.7M dollars). Para la mayoria de las casas (0 - 1M dollars), el modelo permite predecir el precio pero con poca precision.

En caso de que sea afirmativa la respuesta: ¿Con qué
certeza/confianza etc.? Solo 9,6% de todas las predicciones estan a menos de 5% de diferencia del precio real.

En caso de que sea negativa: ¿Por qué no? ¿Se podría mejorar con
más datos u otro método? Probablemente se puede mejorar agregando generations a tpot regressor pero el tiempo de ejecucion podria demorar aun mas que esta ahora.